In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import datetime

In [2]:
expressionDic = {
"0": "angry",
"1" : "disgust",
"2" : "fear",
"3" : "happy",
"4" : "sad",
"5" : "surprise",
"6" : "neutral"}

#incomplete
#https://github.com/tensorflow/tfjs-models/blob/838611c02f51159afdd77469ce67f0e26b7bbb23/face-landmarks-detection/src/mediapipe-facemesh/keypoints.ts
#https://github.com/tensorflow/tfjs-models/blob/646992fd7ab8237c0dc908f2526301414b417c95/face-landmarks-detection/mesh_map.jpg
facialLandmarks = {
  'silhouette': [
    10,  338, 297, 332, 284, 251, 389, 356, 454, 323, 361, 288,
    397, 365, 379, 378, 400, 377, 152, 148, 176, 149, 150, 136,
    172, 58,  132, 93,  234, 127, 162, 21,  54,  103, 67,  109
  ],

  'lipsUpperOuter': [61, 185, 40, 39, 37, 0, 267, 269, 270, 409, 291],
  'lipsLowerOuter': [146, 91, 181, 84, 17, 314, 405, 321, 375, 291],
  'lipsUpperInner': [78, 191, 80, 81, 82, 13, 312, 311, 310, 415, 308],
  'lipsLowerInner': [78, 95, 88, 178, 87, 14, 317, 402, 318, 324, 308],

  'rightEyeUpper0': [246, 161, 160, 159, 158, 157, 173],
  'rightEyeLower0': [33, 7, 163, 144, 145, 153, 154, 155, 133],
  'rightEyeUpper1': [247, 30, 29, 27, 28, 56, 190],
  'rightEyeLower1': [130, 25, 110, 24, 23, 22, 26, 112, 243],
  'rightEyeUpper2': [113, 225, 224, 223, 222, 221, 189],
  'rightEyeLower2': [226, 31, 228, 229, 230, 231, 232, 233, 244],
  'rightEyeLower3': [143, 111, 117, 118, 119, 120, 121, 128, 245],

  'rightEyebrowUpper': [156, 70, 63, 105, 66, 107, 55, 193],
  'rightEyebrowLower': [35, 124, 46, 53, 52, 65],

  'rightEyeIris': [473, 474, 475, 476, 477],

  'leftEyeUpper0': [466, 388, 387, 386, 385, 384, 398],
  'leftEyeLower0': [263, 249, 390, 373, 374, 380, 381, 382, 362],
  'leftEyeUpper1': [467, 260, 259, 257, 258, 286, 414],
  'leftEyeLower1': [359, 255, 339, 254, 253, 252, 256, 341, 463],
  'leftEyeUpper2': [342, 445, 444, 443, 442, 441, 413],
  'leftEyeLower2': [446, 261, 448, 449, 450, 451, 452, 453, 464],
  'leftEyeLower3': [372, 340, 346, 347, 348, 349, 350, 357, 465],

  'leftEyebrowUpper': [383, 300, 293, 334, 296, 336, 285, 417],
  'leftEyebrowLower': [265, 353, 276, 283, 282, 295],

  'leftEyeIris': [468, 469, 470, 471, 472],

  'midwayBetweenEyes': [168],

  'noseTip': [1],
  'noseBottom': [2],
  'noseRightCorner': [98],
  'noseLeftCorner': [327],

  'rightCheek': [205],
  'leftCheek': [425]
}


In [3]:
import math
import numpy as np
def alignNormalizedPointsFixed(xIn : list, yIn: list):
    baseNoseX = xIn[0]
    baseNoseY = yIn[0]

    targetX = .5
    targetY = .5

    deltaX = targetX - baseNoseX
    deltaY = targetY - baseNoseY

    xShift = []
    yShift = []

    for x, y in zip(xIn, yIn):
        xShift.append(x + deltaX)
        yShift.append(y + deltaY) 

    return xShift, yShift


def scalePointsFixed(xIn: list, yIn: list, zIn: list):

    #controlDistBetweenEyes = xCon[243] - xCon[362]
    #hard coding control distance to scale all image to coming from 'same face'
    controlDistBetweenEyes = .05
    testDistBetweenEyes = xIn[243] - xIn[362]

    scale = controlDistBetweenEyes / testDistBetweenEyes

    xOut = list(map(lambda x: x * scale, xIn))
    yOut = list(map(lambda y: y * scale, yIn))
    zOut = list(map(lambda z: z * scale, zIn))
    
    return xOut, yOut, zOut

def flipPoints(input: list):
    flippedPoints = [-p for p in input]
    return flippedPoints

def reversePoints(input: list):
    reveredPoints = [1-p for p in input]
    return reveredPoints

def rotate90CC(affine: np.array):
    rotateMatrix = np.array([[0, 1],
                             [-1, 0]])
    return np.dot(affine, rotateMatrix)

def affineTransform(X: list, Y: list):
    #https://stackoverflow.com/questions/74493141/align-x-and-y-coordinates-of-face-landmarks-in-r
    #133 is left eye tearduct, 362 is right
    #deltaX = X[133] - X[362]
    #deltaY = Y[133] - Y[362]

    #10 is top of forehead, 152 is bottom of chin
    deltaX = X[10] - X[152]
    deltaY = Y[10] - Y[152]
    theta = math.atan2(-deltaY, deltaX)

    rotationMatrix = np.array([[math.cos(theta), -math.sin(theta)],
                              [math.sin(theta), math.cos(theta)]])
    
    rotX = []
    rotY = []

    for x, y in zip(X, Y):
        rotateCoord = np.dot(rotationMatrix, np.array([x, y]))
        rotX.append(rotateCoord[0]) 
        rotY.append(rotateCoord[1])
    
    rotMatrix = np.column_stack((rotX, rotY))

    rotPoints = rotate90CC(rotMatrix)
    rotX = rotPoints[:, 0].tolist()
    rotY = rotPoints[:,1].tolist()

    return rotX, rotY

In [4]:

imageDir = r"C:\Users\myfir\My Drive\Skyline\ExpW\data\data\image\originPad\origin"
labelListPath = r"C:\Users\myfir\My Drive\Skyline\ExpW\data\data\label\label.lst"
headers = ["image_name", "face_id_in_image", "face_box_top" ,"face_box_left" ,"face_box_right" ,"face_box_bottom" ,"face_box_confidence" , "expression_label"]

data = pd.read_csv(labelListPath, delim_whitespace=True, header= None, names= headers, dtype= str)
data['expression_label'] = data['expression_label'].replace(expressionDic)
data= data.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

data = data[['image_name', 'expression_label']]
data = data.drop_duplicates(subset=['image_name'])

In [5]:
# STEP 1: Import the necessary modules.
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

# STEP 2: Create an FaceLandmarker object.
base_options = python.BaseOptions(model_asset_path='face_landmarker.task')
options = vision.FaceLandmarkerOptions(base_options=base_options,
                                       output_face_blendshapes=True,
                                       output_facial_transformation_matrixes=True,
                                       num_faces=1)
detector = vision.FaceLandmarker.create_from_options(options)


In [6]:
import sqlite3
import json

# Connect to the SQLite database (or create it if it doesn't exist)
dbPath = r'C:\Users\myfir\My Drive\Skyline\ExpW\expw.db'
conn = sqlite3.connect(dbPath)
cursor = conn.cursor()

# Create a table with the desired schema
cursor.execute('''
CREATE TABLE IF NOT EXISTS landmarksInput (
    id INTEGER PRIMARY KEY,
    NAME TEXT,
    EXPR TEXT,
    X TEXT,
    Y TEXT,
    Z TEXT
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS landmarksTransform (
    id INTEGER PRIMARY KEY,
    NAME TEXT,
    EXPR TEXT,
    X TEXT,
    Y TEXT,
    Z TEXT
)
''')

# Commit the changes
conn.commit()


dirNames = list(name for name in os.listdir(imageDir) if name.endswith('.jpg'))
imageNames = list(data['image_name'])
bad = []

for name in dirNames:
    if name not in imageNames:
        bad.append(name)

dirNames.sort()
print(repr(bad[0]) == repr(dirNames[5])) #'afraid_american_513.jpg'
print(repr(bad[0])) #'afraid_american_513.jpg'
print(repr(dirNames[5])) #'afraid_american_513.jpg'

print(len((set(list(data['image_name'])))))

# Convert both lists to lowercase for case-insensitive comparison
dirNames_lower = [name.lower() for name in dirNames]
imageNames_lower = [name.lower() for name in imageNames]

intersection_lower = set(dirNames_lower) & set(imageNames_lower)


In [9]:
#filenameList = data['image_name'].tolist()
#labelList = data['expression_label'].tolist()


filenamesList = [name for name in set(os.listdir(imageDir)) if name.endswith('.jpg') and name in set(list(data['image_name']))]



In [10]:


for index, filename in enumerate(filenamesList):
    filepath = os.path.join(imageDir, filename)
    # STEP 3: Load the input image.
    image = mp.Image.create_from_file(filepath)

    # STEP 4: Detect face landmarks from the input image.
    detection_result = detector.detect(image)

    #print(len(detection_result.face_landmarks))
    if len(detection_result.face_landmarks) == 1:
        xs = []
        ys = []
        zs = []
        for face in detection_result.face_landmarks:
            rowIndex = data[data['image_name'] == filename].index[0]
            expr = data.at[rowIndex, 'expression_label']
            for id, landmark in enumerate(face):
                xPure = landmark.x
                yPure = landmark.y
                zPure = landmark.z
                

                xs.append(xPure)
                ys.append(yPure)
                zs.append(zPure)
        xJSON = json.dumps(xs)
        yJSON = json.dumps(ys)
        zJSON = json.dumps(zs)

        # Insert the data into the table
        cursor.execute('''
        INSERT INTO landmarksInput (NAME, EXPR, X, Y, Z)
        VALUES (?, ?, ?, ?, ?)
        ''', (filename, expr, xJSON, yJSON, zJSON))

        # Commit the changes
        conn.commit()


        xs, ys = affineTransform(xs, ys)
        xs, ys, zs = scalePointsFixed(xs, ys, zs)
        xs, ys = alignNormalizedPointsFixed(xs, ys)
    
        xJSON = json.dumps(xs)
        yJSON = json.dumps(ys)
        zJSON = json.dumps(zs)
        # Insert the data into the table
        cursor.execute('''
        INSERT INTO landmarksTransform (NAME, EXPR, X, Y, Z)
        VALUES (?, ?, ?, ?, ?)
        ''', (filename, expr, xJSON, yJSON, zJSON))
        # Commit the changes
        conn.commit()


    if (index+1) % 500 == 0:

        # Get the current date and time
        now = datetime.datetime.now()

        # Format the current time
        formatted_time = now.strftime("%H:%M:%S")

        print(f'Processed {index+1} images at {formatted_time}')
        




c:\Users\myfir\AppData\Local\Programs\Python\Python311\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Processed 500 images at 22:42:39
Processed 1000 images at 22:42:47
Processed 1500 images at 22:42:55
Processed 2000 images at 22:43:02
Processed 2500 images at 22:43:11
Processed 3000 images at 22:43:19
Processed 3500 images at 22:43:28
Processed 4000 images at 22:43:36
Processed 4500 images at 22:43:45
Processed 5000 images at 22:43:53
Processed 5500 images at 22:44:02
Processed 6000 images at 22:44:10
Processed 6500 images at 22:44:19
Processed 7000 images at 22:44:28
Processed 7500 images at 22:44:36
Processed 8000 images at 22:44:44
Processed 8500 images at 22:44:54
Processed 9000 images at 22:45:03
Processed 9500 images at 22:45:12
Processed 10000 images at 22:45:22
Processed 10500 images at 22:45:30
Processed 11000 images at 22:45:39
Processed 11500 images at 22:45:48
Processed 12000 images at 22:45:57
Processed 12500 images at 22:46:05
Processed 13000 images at 22:46:14
Processed 13500 images at 22:46:23
Processed 14000 images at 22:46:32
Processed 14500 images at 22:46:41
Proce

In [ ]:
import statistics

exprs = list(expressionDic.values())

dbPath = r'C:\Users\myfir\My Drive\Skyline\ExpW\expw.db'
conn = sqlite3.connect(dbPath)

query = 'SELECT * FROM landmarksTransform WHERE EXPR = ?'

for expr in exprs:
    cursor = conn.cursor()
    X = np.array()
    Y = np.array()
    Z = np.array()

    coords = [X, Y, Z]
    # Execute a query to retrieve all rows from the table
    cursor.execute(query, (expr,))
    rows = cursor.fetchall()
    for row in rows:
        X = np.concatenate((X, np.array(json.loads(row['X']))))
        Y = np.concatenate((Y, np.array(json.loads(row['Y']))))
        Z = np.concatenate((Z, np.array(json.loads(row['Z']))))

    XSample = X.reshape((-1, 478))
    YSample = Y.reshape((-1, 478))
    ZSample = Z.reshape((-1, 478))

    

    




# Fetch all results and load into a pandas DataFrame
columns = [desc[0] for desc in cursor.description]
rows = cursor.fetchall()
data = pd.DataFrame(rows, columns=columns)

# Close the connection
conn.close()

import ast

# Convert the 'X', 'Y', and 'Z' columns from strings to lists
data['X'] = data['X'].apply(ast.literal_eval)
data['Y'] = data['Y'].apply(ast.literal_eval)
data['Z'] = data['Z'].apply(ast.literal_eval)
